In [137]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import os
import datetime
from datetime import datetime, timedelta
import pytz
from pytz import timezone
import collections
import itertools 
from haversine import haversine

In [2]:
path = 'C:/Python/QuantumBlack/data/'
def read_input_files(path):
    files = []
    df = pd.DataFrame()
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if '.parquet' in file:
                files.append(os.path.join(r, file))
        for f in files:
            df = df.append(pq.read_table(f).to_pandas(),ignore_index=True)
    
    return df

In [3]:
drive_data_orig = read_input_files('C:/Python/QuantumBlack/data/drive')
print(drive_data_orig.shape)
print(drive_data_orig)

(9217528, 12)
         vehicle_id                           trip_id  \
0           1000515  db3c08b01205459294bbc91e3f7dfeb6   
1           1000515  db3c08b01205459294bbc91e3f7dfeb6   
2           1000515  db3c08b01205459294bbc91e3f7dfeb6   
3           1000515  db3c08b01205459294bbc91e3f7dfeb6   
4           1000515  db3c08b01205459294bbc91e3f7dfeb6   
5           1000515  db3c08b01205459294bbc91e3f7dfeb6   
6           1000515  db3c08b01205459294bbc91e3f7dfeb6   
7           1000515  db3c08b01205459294bbc91e3f7dfeb6   
8           1000515  db3c08b01205459294bbc91e3f7dfeb6   
9           1000515  db3c08b01205459294bbc91e3f7dfeb6   
10          1000515  db3c08b01205459294bbc91e3f7dfeb6   
11          1000515  db3c08b01205459294bbc91e3f7dfeb6   
12          1000515  db3c08b01205459294bbc91e3f7dfeb6   
13          1000515  db3c08b01205459294bbc91e3f7dfeb6   
14          1000515  db3c08b01205459294bbc91e3f7dfeb6   
15          1000515  db3c08b01205459294bbc91e3f7dfeb6   
16          10005

In [4]:
#Read trip details
trip_data_orig = read_input_files('C:/Python/QuantumBlack/data/trip')
print(trip_data_orig.shape)
print(trip_data_orig.head(20))

(9217528, 6)
    vehicle_id                           trip_id                  datetime  \
0      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
1      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:01+00:00   
2      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:02+00:00   
3      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:03+00:00   
4      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:04+00:00   
5      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:05+00:00   
6      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:06+00:00   
7      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:07+00:00   
8      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:08+00:00   
9      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:09+00:00   
10     1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:10+00:00   
11     1000504  192637d8a8a841dd85942a5ada12c50a 20

In [5]:
#Read Weather details
weather_data_orig = read_input_files('C:/Python/QuantumBlack/data/weather')
print(weather_data_orig.shape)


(10586016, 14)
      x   y        date time      lat       lon temperature_data  \
0    85  55  2017-02-15  11Z  31.9375 -114.3125       286.430000   
1   191  38  2017-02-23  21Z  29.8125 -101.0625       304.960000   
2   258  60  2017-02-27  16Z  32.5625  -92.6875       287.590000   
3   113  55  2017-01-25  07Z  31.9375 -110.8125       277.800000   
4   304  55  2017-02-03  21Z  31.9375  -86.9375       286.010000   
5   223  48  2017-01-08  03Z  31.0625  -97.0625       270.610000   
6   101  48  2017-02-18  09Z  31.0625 -112.3125       286.900000   
7   130  52  2017-02-13  03Z  31.5625 -108.6875       285.080000   
8   308  59  2017-01-12  02Z  32.4375  -86.4375       290.040000   
9   120  33  2017-01-12  09Z  29.1875 -109.9375       283.070000   
10   80  44  2017-01-19  21Z  30.5625 -114.9375       290.270000   
11  244  47  2017-02-04  03Z  30.9375  -94.4375       282.100000   
12   67  55  2017-02-18  13Z  31.9375 -116.5625       283.400000   
13  267  54  2017-01-05  16Z  31.

In [107]:
weather_data = weather_data_orig.copy()
weather_data = weather_data.sort_values(['lat','lon','date','time'])
print(weather_data.head(50))

           x   y        date time      lat       lon temperature_data  \
10530772  85  30  2017-01-01  00Z  28.8125 -114.3125       289.240000   
5894086   85  30  2017-01-01  01Z  28.8125 -114.3125       288.990000   
7657859   85  30  2017-01-01  02Z  28.8125 -114.3125       288.730000   
2620856   85  30  2017-01-01  03Z  28.8125 -114.3125       288.480000   
8445451   85  30  2017-01-01  04Z  28.8125 -114.3125       288.430000   
9423793   85  30  2017-01-01  05Z  28.8125 -114.3125       288.390000   
5930359   85  30  2017-01-01  06Z  28.8125 -114.3125       288.340000   
4176506   85  30  2017-01-01  07Z  28.8125 -114.3125       288.210000   
5436563   85  30  2017-01-01  08Z  28.8125 -114.3125       288.080000   
2619314   85  30  2017-01-01  09Z  28.8125 -114.3125       287.950000   
8805302   85  30  2017-01-01  10Z  28.8125 -114.3125       287.860000   
4615515   85  30  2017-01-01  11Z  28.8125 -114.3125       287.760000   
1713940   85  30  2017-01-01  12Z  28.8125 -114.312

In [6]:
weather_data_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10586016 entries, 0 to 10586015
Data columns (total 14 columns):
x                     int64
y                     int64
date                  object
time                  object
lat                   float64
lon                   float64
temperature_data      object
temperature_unit      object
precipitation_data    object
precipitation_unit    object
wind_ew_data          object
wind_ew_unit          object
wind_ns_data          object
wind_ns_unit          object
dtypes: float64(2), int64(2), object(10)
memory usage: 1.1+ GB


In [7]:
vehicle_data_orig = pd.read_csv('C:/Python/QuantumBlack/data/vehicle.csv')
print(vehicle_data_orig.shape)
print(vehicle_data_orig.head(20))

(7, 17)
   vehicle_id  year        make        Model  drivetrain  max_torque  \
0     1000500  2016       Honda        Civic           2         174   
1     1000501  2016        Jeep      Compass           4         350   
2     1000502  2016     Hyundai        Creta           2         260   
3     1000503  2016       Skoda       Superb           2         250   
4     1000504  2017  Volkswagen       Passat           4         350   
5     1000506  2017         BMW  3 Series GT           2         400   
6     1000507  2017        Audi           Q3           3         250   

   max_horsepower  max_horsepower_rpm  max_torque_rpm  engine_displacement  \
0             140                6500            1500                1.799   
1             171                3750            1250                1.956   
2             126                4000            1500                1.582   
3             177                5100            1750                1.798   
4             174        

In [8]:
#Put all the data files into excel
#drive_data_orig.to_excel("C:/Python/QuantumBlack/data/drive.xlsx","sheet1")

In [9]:
def convert_time(input_time):
    
    fmt = "%Y-%m-%d"
    date_str=str(input_time)
    d = (datetime.strptime(date_str[0:10], "%Y-%m-%d"))
    return d.strftime(fmt)
    
    #print(input_time)
    #date_str=str(input_time)
    #datetime_obj = datetime.strptime(date_str[0:10], "%Y-%m-%d")
    #datetime_obj_utc = datetime_obj.replace(tzinfo=timezone('UTC'))
    #     date = datetime(input_time)
    #     utc = pytz.UTC
    #     pst = pytz.timezone('America/Los_Angeles')
    #     d = utc.localize(d) # UTC timezone aware
    #     d = d.astimezone(pst)
    
    #print(d)
    
    #     datetime_obj_utc = pytz.utc.localize(datetime_obj)
    #     print(datetime_obj_utc)
    #     pst = datetime_obj_utc.astimezone(pytz.timezone("America/Los_Angeles"))
    #     print(pst.strftime(fmt))
    #return pst.strftime(fmt)
    
    #pst = input_time.dt.tz_localize('GMT').dt.tz_convert('America/Los_Angeles')
    #print(pst.strftime(fmt))
    #d_pacific_tz_aware = input_time.datetime.tz_localize("GMT").dt.tz_convert('America/Los_Angeles')
    #print(d_pacific_tz_aware)
    #d_pacific_tz_naive = input_time.datetime.tz_localize("GMT").dt.tz_convert('America/Los_Angeles').dt.tz_localize(None)
    #print(d_pacific_tz_naive)
    #return pst
    #utc_now = pytz.utc.localize(input_time)
    #pst_now = utc_now.astimezone(pytz.timezone("America/Los_Angeles"))
    #print(pst_now)

Start working on creating Engine_features.csv

In [10]:
#check whether there are null values in drivedata
print(drive_data_orig.isnull().sum())

vehicle_id             0
trip_id                0
datetime               0
velocity               0
accel_x                0
accel_y                0
accel_z                0
engine_coolant_temp    0
eng_load               0
fuel_level             0
iat                    0
rpm                    0
dtype: int64


In [11]:
drive_data_orig.describe()

,vehicle_id,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm
count,9.217528e+06,9.217528e+06,9.217528e+06,9.217528e+06,9.217528e+06,9.217528e+06,9.217528e+06,9.217528e+06,9.217528e+06,9.217528e+06
mean,1.000509e+06,6.440624e+01,6.319614e+01,6.395792e+01,6.430437e+01,1.267643e+02,2.044824e+02,1.199065e+02,9.314448e+01,2.049561e+03
std,5.789579e+00,1.322765e+01,1.719181e+01,1.700744e+01,1.715290e+01,3.388958e+01,1.001103e+01,3.846573e+01,2.953116e+01,8.486932e+01
min,1.000500e+06,-1.270000e+01,2.300000e+01,2.400000e+01,2.300000e+01,5.500000e+01,1.665700e+02,3.800000e+01,2.800000e+01,1.879480e+03
25%,1.000504e+06,5.533000e+01,4.700000e+01,4.800000e+01,4.800000e+01,9.500000e+01,1.968700e+02,8.900000e+01,6.900000e+01,1.977900e+03
50%,1.000509e+06,6.433000e+01,5.900000e+01,6.400000e+01,6.627000e+01,1.130000e+02,2.043600e+02,1.130000e+02,8.800000e+01,2.051060e+03
75%,1.000514e+06,7.345000e+01,7.924000e+01,7.953000e+01,7.987000e+01,1.592600e+02,2.121300e+02,1.489800e+02,1.175000e+02,2.124540e+03
max,1.000519e+06,1.413400e+02,1.129700e+02,1.118600e+02,1.114800e+02,1.994600e+02,2.416700e+02,2.209400e+02,1.707300e+02,2.220480e+03


In [67]:
drive_data = drive_data_orig.copy()

In [68]:
drive_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9217528 entries, 0 to 9217527
Data columns (total 12 columns):
vehicle_id             int64
trip_id                object
datetime               datetime64[ns, UTC]
velocity               float64
accel_x                float64
accel_y                float64
accel_z                float64
engine_coolant_temp    float64
eng_load               float64
fuel_level             float64
iat                    float64
rpm                    float64
dtypes: datetime64[ns, UTC](1), float64(9), int64(1), object(1)
memory usage: 843.9+ MB


In [69]:
drive_data['date'] = drive_data['datetime'] - timedelta(hours=7)
drive_data['date']=np.vectorize(convert_time)(drive_data['date'])

In [70]:
drive_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9217528 entries, 0 to 9217527
Data columns (total 13 columns):
vehicle_id             int64
trip_id                object
datetime               datetime64[ns, UTC]
velocity               float64
accel_x                float64
accel_y                float64
accel_z                float64
engine_coolant_temp    float64
eng_load               float64
fuel_level             float64
iat                    float64
rpm                    float64
date                   object
dtypes: datetime64[ns, UTC](1), float64(9), int64(1), object(2)
memory usage: 914.2+ MB


In [16]:
engine_features = pd.DataFrame()

In [17]:
engine_features = pd.merge(drive_data,vehicle_data_orig,how='left',left_on='vehicle_id',right_on='vehicle_id')
print(engine_features.head())

   vehicle_id                           trip_id                  datetime  \
0     1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:00:00+00:00   
1     1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:00:01+00:00   
2     1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:00:02+00:00   
3     1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:00:03+00:00   
4     1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:00:04+00:00   

   velocity  accel_x  accel_y  accel_z  engine_coolant_temp  eng_load  \
0      0.00     50.0    65.13    87.92               150.70    192.88   
1     47.49     51.0    72.34    83.78               154.76    196.04   
2     57.94     50.0    69.20    87.73               165.50    196.01   
3     57.50     48.0    70.83    84.95               157.32    193.31   
4     52.29     47.0    70.22    92.90               151.00    189.49   

   fuel_level  ...  max_horsepower_rpm  max_torque_rpm engine_displacement  \
0      118.66  ...  

In [18]:
print(engine_features.shape)

(9217528, 29)


In [19]:
engine_features = engine_features.fillna(0)

In [20]:
engine_features['active_horsepower'] = (engine_features['eng_load']/255) * engine_features['max_torque'] * (engine_features['rpm']/5252)

In [21]:

engine_features['horsepower_util'] = engine_features['active_horsepower'] / engine_features['max_horsepower'] 
#If max_horsepower is 0,  it makes horsepower_util to infinity. Hence replacing inf with 0s
engine_features['horsepower_util'] = engine_features['horsepower_util'].replace([np.inf, -np.inf], 0)
engine_features['horsepower_util'] = engine_features['horsepower_util'].fillna(0)


In [22]:
engine_features['Torque_util'] = engine_features['eng_load']/255

In [23]:
engine_features['rpm_util'] = engine_features['rpm']/engine_features['max_horsepower_rpm']
#If max_horsepower is 0,  it makes rpm_util to infinity. Hence replacing inf with 0s

engine_features['rpm_util'] = engine_features['rpm_util'].replace([np.inf, -np.inf], 0)
print(engine_features['max_horsepower_rpm'].unique())

[   0. 4000. 3600. 5000. 5100. 3750. 6500.]


In [24]:
#Creating buckets for torqueutil
torque_bucket = pd.DataFrame()
torque_bins = np.array([0.60,0.70,0.80,0.90,1.0])
torque_bucket = pd.cut(engine_features['Torque_util'], torque_bins,labels=["60pct_s", "70pct_s", "80pct_s","90pct_s"],include_lowest=True)
    

In [25]:
np.histogram(engine_features['Torque_util'], torque_bins)

(array([   7578, 4487316, 4708994,   13640], dtype=int64),
 array([0.6, 0.7, 0.8, 0.9, 1. ]))

In [26]:
ft_torque_util = pd.get_dummies(torque_bucket,prefix='ft_torque_util',prefix_sep='_')


In [27]:
engine_features = pd.concat([engine_features,ft_torque_util],axis=1)
print(engine_features.head(20))

    vehicle_id                           trip_id                  datetime  \
0      1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:00:00+00:00   
1      1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:00:01+00:00   
2      1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:00:02+00:00   
3      1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:00:03+00:00   
4      1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:00:04+00:00   
5      1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:00:05+00:00   
6      1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:00:06+00:00   
7      1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:00:07+00:00   
8      1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:00:08+00:00   
9      1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:00:09+00:00   
10     1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:00:10+00:00   
11     1000515  db3c08b01205459294bbc91e3f7dfeb6 2017-01-06 23:0

In [28]:
print(engine_features['rpm_util'].unique())

[0.         0.5159375  0.515195   ... 0.52478333 0.52602222 0.52691389]


In [29]:
#Creating buckets for horsepower util
horsepower_bucket = pd.DataFrame()
horsepower_bins = np.array([0.0,0.50,0.60,0.70,0.80,0.90])
horsepower_bucket = pd.cut(engine_features['horsepower_util'], horsepower_bins,include_lowest=True,labels=["0pct_s","50pct_s", "60pct_s", "70pct_s","80pct_s"])
#horsepower_bucket = horsepower_bucket.fillna(0)


In [30]:
np.histogram(engine_features['horsepower_util'], horsepower_bins)

(array([6967623,  685153, 1379586,  185142,      24], dtype=int64),
 array([0. , 0.5, 0.6, 0.7, 0.8, 0.9]))

In [31]:
ft_horsepower_util = pd.get_dummies(horsepower_bucket,prefix='ft_horsepower_util',prefix_sep='_')


In [32]:
engine_features = pd.concat([engine_features,ft_horsepower_util],axis=1)


In [33]:
#Creating buckets for rpm util
rpm_bucket = pd.DataFrame()
rpm_bins = np.array([0.0,0.50,0.60,0.70])
rpm_bucket = pd.cut(engine_features['rpm_util'], rpm_bins,labels=["0pct_s","50pct_s", "60pct_s"],include_lowest=True)


In [34]:
print(rpm_bucket.unique())

[0pct_s, 50pct_s, 60pct_s]
Categories (3, object): [0pct_s < 50pct_s < 60pct_s]


In [35]:
ft_rpm_util = pd.get_dummies(rpm_bucket,prefix='ft_rpm_util',prefix_sep='_')

In [36]:
engine_features = pd.concat([engine_features,ft_rpm_util],axis=1)


Do a groupby using the vehicle ID and week and take a sum of each .Week starts from a Monday 

In [37]:
print(engine_features.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9217528 entries, 0 to 9217527
Data columns (total 45 columns):
vehicle_id                    int64
trip_id                       object
datetime                      datetime64[ns, UTC]
velocity                      float64
accel_x                       float64
accel_y                       float64
accel_z                       float64
engine_coolant_temp           float64
eng_load                      float64
fuel_level                    float64
iat                           float64
rpm                           float64
date                          object
year                          float64
make                          object
Model                         object
drivetrain                    float64
max_torque                    float64
max_horsepower                float64
max_horsepower_rpm            float64
max_torque_rpm                float64
engine_displacement           float64
fuel_type                     float64
fuel_ta

In [38]:
df = engine_features.groupby(['vehicle_id', pd.Grouper(key='datetime', freq='W-Mon',sort=True)])

In [39]:

f = collections.OrderedDict({'ft_torque_util_60pct_s':['sum'],'ft_torque_util_70pct_s':['sum'],'ft_torque_util_80pct_s':['sum'],'ft_torque_util_90pct_s':['sum'],'ft_horsepower_util_50pct_s':['sum'],'ft_horsepower_util_60pct_s':['sum'],'ft_horsepower_util_70pct_s':['sum'],'ft_horsepower_util_80pct_s':['sum'],'ft_rpm_util_50pct_s':['sum'],'ft_rpm_util_60pct_s':['sum']})
features = df.agg(f)

In [41]:
features.to_csv (r'C:/Python/QuantumBlack/data/engine_features.csv',index_label=['vehicle_id','week_start_date'],header=['ft_torque_util_60pct_s','ft_torque_util_70pct_s','ft_torque_util_80pct_s','ft_torque_util_90pct_s','ft_horsepower_util_50pct_s','ft_horsepower_util_60pct_s','ft_horsepower_util_70pct_s','ft_horsepower_util_80pct_s','ft_rpm_util_50pct_s','ft_rpm_util_60pct_s'])

Working on drive_features

Converting velocity from km/h to m/s to match acceleration in the final table
To convert km/hr into m/sec, multiply the number by 5 and then divide it by 18.

In [71]:
drive_data['velocityms'] = drive_data['velocity'] *5/18

Ensure the data is sorted for calculating acceleration and deceleration

In [72]:
drive_data_sort = drive_data.sort_values(['vehicle_id','trip_id','datetime'])

In [73]:
drive_data_group = drive_data_sort.groupby(['vehicle_id','trip_id','date'])


Calculate difference in velocity

In [ ]:
#gb.apply(lambda _df: _df.sort_values(by=['id'])
#drive_data_group.apply(lamda drive_data_group['diff_velocity']:drive_data_group['velocity'].diff())

In [65]:
# drive_data_sort['diff_velocity'] = drive_data_sort['diff_velocity'].fillna(0)
# drive_data_sort.head(20)

calculate difference in time

In [74]:
drive_data_sort['diff_time'] = drive_data_sort['datetime'].diff()
drive_data_sort['diff_time'] = list(map(lambda x: x.total_seconds(),drive_data_sort['diff_time']))

calculate acceleration as change in velocity / change in time

In [ ]:
# drive_data_sort['decel_lag'] = drive_data_sort['calc_accel'].shift(1)

In [ ]:

#drive_data_sort['decel_win'] = list(map(lambda x: (x['deceleration'] == 1 & x['decel_lag'] == 0.0),drive_data_sort))

In [ ]:
#drive_data_sort.head(50)

In [76]:
drive_data_sort['vel_lag'] = drive_data_sort.groupby(['vehicle_id','trip_id'])['velocityms'].shift(1)

In [77]:
drive_data_sort['diff_velocity'] = drive_data_sort['vel_lag']-drive_data_sort['velocityms']
drive_data_sort['calc_accel'] = drive_data_sort['diff_velocity']/drive_data_sort['diff_time']
drive_data_sort.head(50)

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm,date,velocityms,diff_time,vel_lag,diff_velocity,calc_accel
7482371,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:00+00:00,0.00,33.0,47.0,74.99,77.0,194.15,129.01,59.0,1913.42,2017-01-01,0.000000,NaN,NaN,NaN,NaN
7482372,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:01+00:00,82.68,38.0,45.0,72.44,87.0,198.85,131.83,71.0,1918.20,2017-01-01,22.966667,1.0,0.000000,-22.966667,-22.966667
7482373,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:02+00:00,74.09,38.0,48.0,75.17,80.0,204.14,133.15,68.0,1929.89,2017-01-01,20.580556,1.0,22.966667,2.386111,2.386111
7482374,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:03+00:00,94.06,47.0,42.0,72.80,80.0,200.82,126.37,71.0,1929.39,2017-01-01,26.127778,1.0,20.580556,-5.547222,-5.547222
7482375,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:04+00:00,81.96,40.0,51.0,73.86,85.0,205.90,133.60,65.0,1926.42,2017-01-01,22.766667,1.0,26.127778,3.361111,3.361111
7482376,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:05+00:00,90.63,38.0,49.0,77.98,91.0,194.21,125.53,68.0,1925.06,2017-01-01,25.175000,1.0,22.766667,-2.408333,-2.408333
7482377,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:06+00:00,66.66,47.0,41.0,67.28,77.0,201.01,121.97,69.0,1934.74,2017-01-01,18.516667,1.0,25.175000,6.658333,6.658333
7482378,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:07+00:00,56.10,36.0,39.0,70.54,89.0,207.53,125.70,66.0,1925.81,2017-01-01,15.583333,1.0,18.516667,2.933333,2.933333
7482379,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:08+00:00,79.38,47.0,46.0,79.74,85.0,201.61,122.27,64.0,1927.12,2017-01-01,22.050000,1.0,15.583333,-6.466667,-6.466667
7482380,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:09+00:00,79.75,32.0,49.0,73.20,93.0,202.81,131.68,69.0,1927.56,2017-01-01,22.152778,1.0,22.050000,-0.102778,-0.102778


In [78]:
drive_data_sort['deceleration'] = list(map(lambda x: 1 if (x < 0)  else 0,drive_data_sort['calc_accel']))
drive_data_sort.head(50)

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm,date,velocityms,diff_time,vel_lag,diff_velocity,calc_accel,deceleration
7482371,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:00+00:00,0.00,33.0,47.0,74.99,77.0,194.15,129.01,59.0,1913.42,2017-01-01,0.000000,NaN,NaN,NaN,NaN,0
7482372,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:01+00:00,82.68,38.0,45.0,72.44,87.0,198.85,131.83,71.0,1918.20,2017-01-01,22.966667,1.0,0.000000,-22.966667,-22.966667,1
7482373,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:02+00:00,74.09,38.0,48.0,75.17,80.0,204.14,133.15,68.0,1929.89,2017-01-01,20.580556,1.0,22.966667,2.386111,2.386111,0
7482374,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:03+00:00,94.06,47.0,42.0,72.80,80.0,200.82,126.37,71.0,1929.39,2017-01-01,26.127778,1.0,20.580556,-5.547222,-5.547222,1
7482375,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:04+00:00,81.96,40.0,51.0,73.86,85.0,205.90,133.60,65.0,1926.42,2017-01-01,22.766667,1.0,26.127778,3.361111,3.361111,0
7482376,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:05+00:00,90.63,38.0,49.0,77.98,91.0,194.21,125.53,68.0,1925.06,2017-01-01,25.175000,1.0,22.766667,-2.408333,-2.408333,1
7482377,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:06+00:00,66.66,47.0,41.0,67.28,77.0,201.01,121.97,69.0,1934.74,2017-01-01,18.516667,1.0,25.175000,6.658333,6.658333,0
7482378,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:07+00:00,56.10,36.0,39.0,70.54,89.0,207.53,125.70,66.0,1925.81,2017-01-01,15.583333,1.0,18.516667,2.933333,2.933333,0
7482379,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:08+00:00,79.38,47.0,46.0,79.74,85.0,201.61,122.27,64.0,1927.12,2017-01-01,22.050000,1.0,15.583333,-6.466667,-6.466667,1
7482380,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:09+00:00,79.75,32.0,49.0,73.20,93.0,202.81,131.68,69.0,1927.56,2017-01-01,22.152778,1.0,22.050000,-0.102778,-0.102778,1


In [79]:
drive_data_sort['d_lag'] = drive_data_sort.groupby(['vehicle_id','trip_id','date'])['deceleration'].shift(-1)
drive_data_sort.head(50)

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm,date,velocityms,diff_time,vel_lag,diff_velocity,calc_accel,deceleration,d_lag
7482371,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:00+00:00,0.00,33.0,47.0,74.99,77.0,194.15,129.01,59.0,1913.42,2017-01-01,0.000000,NaN,NaN,NaN,NaN,0,1.0
7482372,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:01+00:00,82.68,38.0,45.0,72.44,87.0,198.85,131.83,71.0,1918.20,2017-01-01,22.966667,1.0,0.000000,-22.966667,-22.966667,1,0.0
7482373,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:02+00:00,74.09,38.0,48.0,75.17,80.0,204.14,133.15,68.0,1929.89,2017-01-01,20.580556,1.0,22.966667,2.386111,2.386111,0,1.0
7482374,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:03+00:00,94.06,47.0,42.0,72.80,80.0,200.82,126.37,71.0,1929.39,2017-01-01,26.127778,1.0,20.580556,-5.547222,-5.547222,1,0.0
7482375,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:04+00:00,81.96,40.0,51.0,73.86,85.0,205.90,133.60,65.0,1926.42,2017-01-01,22.766667,1.0,26.127778,3.361111,3.361111,0,1.0
7482376,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:05+00:00,90.63,38.0,49.0,77.98,91.0,194.21,125.53,68.0,1925.06,2017-01-01,25.175000,1.0,22.766667,-2.408333,-2.408333,1,0.0
7482377,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:06+00:00,66.66,47.0,41.0,67.28,77.0,201.01,121.97,69.0,1934.74,2017-01-01,18.516667,1.0,25.175000,6.658333,6.658333,0,0.0
7482378,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:07+00:00,56.10,36.0,39.0,70.54,89.0,207.53,125.70,66.0,1925.81,2017-01-01,15.583333,1.0,18.516667,2.933333,2.933333,0,1.0
7482379,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:08+00:00,79.38,47.0,46.0,79.74,85.0,201.61,122.27,64.0,1927.12,2017-01-01,22.050000,1.0,15.583333,-6.466667,-6.466667,1,1.0
7482380,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:09+00:00,79.75,32.0,49.0,73.20,93.0,202.81,131.68,69.0,1927.56,2017-01-01,22.152778,1.0,22.050000,-0.102778,-0.102778,1,0.0


In [80]:
def calculate_dcount(x,y):
    if (x==0 and y==1):
        return 1
    else:
        return 0

In [81]:
drive_data_sort['d_count'] = np.vectorize(calculate_dcount)(drive_data_sort['deceleration'],drive_data_sort['d_lag'])

In [78]:
drive_data_sort.head(50)

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,...,rpm,date,diff_velocity,diff_time,calc_accel,deceleration,vel_lag,d_lag,d_count,a_count
7482371,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:00+00:00,0.000000,33.0,47.0,74.99,77.0,194.15,129.01,...,1913.42,2017-01-01,NaN,NaN,NaN,0,NaN,1.0,1,1
7482372,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:01+00:00,6.379630,38.0,45.0,72.44,87.0,198.85,131.83,...,1918.20,2017-01-01,-6.379630,1.0,-6.379630,1,0.000000,0.0,0,0
7482373,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:02+00:00,5.716821,38.0,48.0,75.17,80.0,204.14,133.15,...,1929.89,2017-01-01,0.662809,1.0,0.662809,0,6.379630,1.0,1,1
7482374,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:03+00:00,7.257716,47.0,42.0,72.80,80.0,200.82,126.37,...,1929.39,2017-01-01,-1.540895,1.0,-1.540895,1,5.716821,0.0,0,0
7482375,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:04+00:00,6.324074,40.0,51.0,73.86,85.0,205.90,133.60,...,1926.42,2017-01-01,0.933642,1.0,0.933642,0,7.257716,1.0,1,1
7482376,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:05+00:00,6.993056,38.0,49.0,77.98,91.0,194.21,125.53,...,1925.06,2017-01-01,-0.668981,1.0,-0.668981,1,6.324074,0.0,0,0
7482377,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:06+00:00,5.143519,47.0,41.0,67.28,77.0,201.01,121.97,...,1934.74,2017-01-01,1.849537,1.0,1.849537,0,6.993056,0.0,0,0
7482378,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:07+00:00,4.328704,36.0,39.0,70.54,89.0,207.53,125.70,...,1925.81,2017-01-01,0.814815,1.0,0.814815,0,5.143519,1.0,1,1
7482379,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:08+00:00,6.125000,47.0,46.0,79.74,85.0,201.61,122.27,...,1927.12,2017-01-01,-1.796296,1.0,-1.796296,1,4.328704,1.0,0,0
7482380,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:09+00:00,6.153549,32.0,49.0,73.20,93.0,202.81,131.68,...,1927.56,2017-01-01,-0.028549,1.0,-0.028549,1,6.125000,0.0,0,0


In [82]:
def calculate_acount(x,y):
    if (x==1 and y==0):
        return 1
    else:
        return 0

In [83]:
drive_data_sort['a_count'] = np.vectorize(calculate_acount)(drive_data_sort['deceleration'],drive_data_sort['d_lag'])
drive_data_sort.head(50)

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,...,date,velocityms,diff_time,vel_lag,diff_velocity,calc_accel,deceleration,d_lag,d_count,a_count
7482371,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:00+00:00,0.00,33.0,47.0,74.99,77.0,194.15,129.01,...,2017-01-01,0.000000,NaN,NaN,NaN,NaN,0,1.0,1,0
7482372,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:01+00:00,82.68,38.0,45.0,72.44,87.0,198.85,131.83,...,2017-01-01,22.966667,1.0,0.000000,-22.966667,-22.966667,1,0.0,0,1
7482373,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:02+00:00,74.09,38.0,48.0,75.17,80.0,204.14,133.15,...,2017-01-01,20.580556,1.0,22.966667,2.386111,2.386111,0,1.0,1,0
7482374,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:03+00:00,94.06,47.0,42.0,72.80,80.0,200.82,126.37,...,2017-01-01,26.127778,1.0,20.580556,-5.547222,-5.547222,1,0.0,0,1
7482375,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:04+00:00,81.96,40.0,51.0,73.86,85.0,205.90,133.60,...,2017-01-01,22.766667,1.0,26.127778,3.361111,3.361111,0,1.0,1,0
7482376,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:05+00:00,90.63,38.0,49.0,77.98,91.0,194.21,125.53,...,2017-01-01,25.175000,1.0,22.766667,-2.408333,-2.408333,1,0.0,0,1
7482377,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:06+00:00,66.66,47.0,41.0,67.28,77.0,201.01,121.97,...,2017-01-01,18.516667,1.0,25.175000,6.658333,6.658333,0,0.0,0,0
7482378,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:07+00:00,56.10,36.0,39.0,70.54,89.0,207.53,125.70,...,2017-01-01,15.583333,1.0,18.516667,2.933333,2.933333,0,1.0,1,0
7482379,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:08+00:00,79.38,47.0,46.0,79.74,85.0,201.61,122.27,...,2017-01-01,22.050000,1.0,15.583333,-6.466667,-6.466667,1,1.0,0,0
7482380,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:09+00:00,79.75,32.0,49.0,73.20,93.0,202.81,131.68,...,2017-01-01,22.152778,1.0,22.050000,-0.102778,-0.102778,1,0.0,0,1


In [84]:
def calculate_brake(x,y,z):
    if (x==1) and (y==1) and (z==0):
        return 1
    else:
        return 0
    

In [85]:
drive_data_sort['brake'] = np.vectorize(calculate_brake)(drive_data_sort['deceleration'],drive_data_sort['d_lag'],drive_data_sort['d_count'])
drive_data_sort.head(100)

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,...,velocityms,diff_time,vel_lag,diff_velocity,calc_accel,deceleration,d_lag,d_count,a_count,brake
7482371,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:00+00:00,0.00,33.0,47.0,74.99,77.0,194.15,129.01,...,0.000000,NaN,NaN,NaN,NaN,0,1.0,1,0,0
7482372,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:01+00:00,82.68,38.0,45.0,72.44,87.0,198.85,131.83,...,22.966667,1.0,0.000000,-22.966667,-22.966667,1,0.0,0,1,0
7482373,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:02+00:00,74.09,38.0,48.0,75.17,80.0,204.14,133.15,...,20.580556,1.0,22.966667,2.386111,2.386111,0,1.0,1,0,0
7482374,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:03+00:00,94.06,47.0,42.0,72.80,80.0,200.82,126.37,...,26.127778,1.0,20.580556,-5.547222,-5.547222,1,0.0,0,1,0
7482375,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:04+00:00,81.96,40.0,51.0,73.86,85.0,205.90,133.60,...,22.766667,1.0,26.127778,3.361111,3.361111,0,1.0,1,0,0
7482376,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:05+00:00,90.63,38.0,49.0,77.98,91.0,194.21,125.53,...,25.175000,1.0,22.766667,-2.408333,-2.408333,1,0.0,0,1,0
7482377,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:06+00:00,66.66,47.0,41.0,67.28,77.0,201.01,121.97,...,18.516667,1.0,25.175000,6.658333,6.658333,0,0.0,0,0,0
7482378,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:07+00:00,56.10,36.0,39.0,70.54,89.0,207.53,125.70,...,15.583333,1.0,18.516667,2.933333,2.933333,0,1.0,1,0,0
7482379,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:08+00:00,79.38,47.0,46.0,79.74,85.0,201.61,122.27,...,22.050000,1.0,15.583333,-6.466667,-6.466667,1,1.0,0,0,1
7482380,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:09+00:00,79.75,32.0,49.0,73.20,93.0,202.81,131.68,...,22.152778,1.0,22.050000,-0.102778,-0.102778,1,0.0,0,1,0


In [86]:
def calculate_hard_brake_10(x,value):
    if (x==1 and value <= -10):
        return 1
    else:
        return 0

In [87]:
drive_data_sort['hard_brake_10'] = np.vectorize(calculate_hard_brake_10)(drive_data_sort['brake'],drive_data_sort['calc_accel'])
drive_data_sort.head(100)

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,...,diff_time,vel_lag,diff_velocity,calc_accel,deceleration,d_lag,d_count,a_count,brake,hard_brake_10
7482371,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:00+00:00,0.00,33.0,47.0,74.99,77.0,194.15,129.01,...,NaN,NaN,NaN,NaN,0,1.0,1,0,0,0
7482372,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:01+00:00,82.68,38.0,45.0,72.44,87.0,198.85,131.83,...,1.0,0.000000,-22.966667,-22.966667,1,0.0,0,1,0,0
7482373,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:02+00:00,74.09,38.0,48.0,75.17,80.0,204.14,133.15,...,1.0,22.966667,2.386111,2.386111,0,1.0,1,0,0,0
7482374,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:03+00:00,94.06,47.0,42.0,72.80,80.0,200.82,126.37,...,1.0,20.580556,-5.547222,-5.547222,1,0.0,0,1,0,0
7482375,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:04+00:00,81.96,40.0,51.0,73.86,85.0,205.90,133.60,...,1.0,26.127778,3.361111,3.361111,0,1.0,1,0,0,0
7482376,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:05+00:00,90.63,38.0,49.0,77.98,91.0,194.21,125.53,...,1.0,22.766667,-2.408333,-2.408333,1,0.0,0,1,0,0
7482377,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:06+00:00,66.66,47.0,41.0,67.28,77.0,201.01,121.97,...,1.0,25.175000,6.658333,6.658333,0,0.0,0,0,0,0
7482378,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:07+00:00,56.10,36.0,39.0,70.54,89.0,207.53,125.70,...,1.0,18.516667,2.933333,2.933333,0,1.0,1,0,0,0
7482379,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:08+00:00,79.38,47.0,46.0,79.74,85.0,201.61,122.27,...,1.0,15.583333,-6.466667,-6.466667,1,1.0,0,0,1,0
7482380,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:09+00:00,79.75,32.0,49.0,73.20,93.0,202.81,131.68,...,1.0,22.050000,-0.102778,-0.102778,1,0.0,0,1,0,0


In [88]:
def calculate_hard_brake_3(x,value):
    if (x==1 and value <= -3 and value > -10):
        return 1
    else:
        return 0

In [89]:
drive_data_sort['hard_brake_3'] = np.vectorize(calculate_hard_brake_3)(drive_data_sort['brake'],drive_data_sort['calc_accel'])
drive_data_sort.head(100)

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,...,vel_lag,diff_velocity,calc_accel,deceleration,d_lag,d_count,a_count,brake,hard_brake_10,hard_brake_3
7482371,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:00+00:00,0.00,33.0,47.0,74.99,77.0,194.15,129.01,...,NaN,NaN,NaN,0,1.0,1,0,0,0,0
7482372,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:01+00:00,82.68,38.0,45.0,72.44,87.0,198.85,131.83,...,0.000000,-22.966667,-22.966667,1,0.0,0,1,0,0,0
7482373,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:02+00:00,74.09,38.0,48.0,75.17,80.0,204.14,133.15,...,22.966667,2.386111,2.386111,0,1.0,1,0,0,0,0
7482374,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:03+00:00,94.06,47.0,42.0,72.80,80.0,200.82,126.37,...,20.580556,-5.547222,-5.547222,1,0.0,0,1,0,0,0
7482375,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:04+00:00,81.96,40.0,51.0,73.86,85.0,205.90,133.60,...,26.127778,3.361111,3.361111,0,1.0,1,0,0,0,0
7482376,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:05+00:00,90.63,38.0,49.0,77.98,91.0,194.21,125.53,...,22.766667,-2.408333,-2.408333,1,0.0,0,1,0,0,0
7482377,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:06+00:00,66.66,47.0,41.0,67.28,77.0,201.01,121.97,...,25.175000,6.658333,6.658333,0,0.0,0,0,0,0,0
7482378,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:07+00:00,56.10,36.0,39.0,70.54,89.0,207.53,125.70,...,18.516667,2.933333,2.933333,0,1.0,1,0,0,0,0
7482379,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:08+00:00,79.38,47.0,46.0,79.74,85.0,201.61,122.27,...,15.583333,-6.466667,-6.466667,1,1.0,0,0,1,0,1
7482380,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:09+00:00,79.75,32.0,49.0,73.20,93.0,202.81,131.68,...,22.050000,-0.102778,-0.102778,1,0.0,0,1,0,0,0


In [90]:
def calculate_accel(x,y,z):
    if (x==0) and (y==0) and (z==0):
        return 1
    else:
        return 0

In [91]:
drive_data_sort['accel'] = np.vectorize(calculate_accel)(drive_data_sort['deceleration'],drive_data_sort['d_lag'],drive_data_sort['a_count'])
drive_data_sort.head(100)

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,...,diff_velocity,calc_accel,deceleration,d_lag,d_count,a_count,brake,hard_brake_10,hard_brake_3,accel
7482371,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:00+00:00,0.00,33.0,47.0,74.99,77.0,194.15,129.01,...,NaN,NaN,0,1.0,1,0,0,0,0,0
7482372,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:01+00:00,82.68,38.0,45.0,72.44,87.0,198.85,131.83,...,-22.966667,-22.966667,1,0.0,0,1,0,0,0,0
7482373,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:02+00:00,74.09,38.0,48.0,75.17,80.0,204.14,133.15,...,2.386111,2.386111,0,1.0,1,0,0,0,0,0
7482374,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:03+00:00,94.06,47.0,42.0,72.80,80.0,200.82,126.37,...,-5.547222,-5.547222,1,0.0,0,1,0,0,0,0
7482375,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:04+00:00,81.96,40.0,51.0,73.86,85.0,205.90,133.60,...,3.361111,3.361111,0,1.0,1,0,0,0,0,0
7482376,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:05+00:00,90.63,38.0,49.0,77.98,91.0,194.21,125.53,...,-2.408333,-2.408333,1,0.0,0,1,0,0,0,0
7482377,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:06+00:00,66.66,47.0,41.0,67.28,77.0,201.01,121.97,...,6.658333,6.658333,0,0.0,0,0,0,0,0,1
7482378,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:07+00:00,56.10,36.0,39.0,70.54,89.0,207.53,125.70,...,2.933333,2.933333,0,1.0,1,0,0,0,0,0
7482379,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:08+00:00,79.38,47.0,46.0,79.74,85.0,201.61,122.27,...,-6.466667,-6.466667,1,1.0,0,0,1,0,1,0
7482380,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:09+00:00,79.75,32.0,49.0,73.20,93.0,202.81,131.68,...,-0.102778,-0.102778,1,0.0,0,1,0,0,0,0


In [92]:
def calculate_hard_accel_10(x,value):
    if (x==1 and value >= 10):
        return 1
    else:
        return 0

In [93]:
drive_data_sort['hard_accel_10'] = np.vectorize(calculate_hard_accel_10)(drive_data_sort['accel'],drive_data_sort['calc_accel'])
drive_data_sort.head(100)

C:\Users\Srividya\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,...,calc_accel,deceleration,d_lag,d_count,a_count,brake,hard_brake_10,hard_brake_3,accel,hard_accel_10
7482371,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:00+00:00,0.00,33.0,47.0,74.99,77.0,194.15,129.01,...,NaN,0,1.0,1,0,0,0,0,0,0
7482372,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:01+00:00,82.68,38.0,45.0,72.44,87.0,198.85,131.83,...,-22.966667,1,0.0,0,1,0,0,0,0,0
7482373,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:02+00:00,74.09,38.0,48.0,75.17,80.0,204.14,133.15,...,2.386111,0,1.0,1,0,0,0,0,0,0
7482374,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:03+00:00,94.06,47.0,42.0,72.80,80.0,200.82,126.37,...,-5.547222,1,0.0,0,1,0,0,0,0,0
7482375,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:04+00:00,81.96,40.0,51.0,73.86,85.0,205.90,133.60,...,3.361111,0,1.0,1,0,0,0,0,0,0
7482376,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:05+00:00,90.63,38.0,49.0,77.98,91.0,194.21,125.53,...,-2.408333,1,0.0,0,1,0,0,0,0,0
7482377,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:06+00:00,66.66,47.0,41.0,67.28,77.0,201.01,121.97,...,6.658333,0,0.0,0,0,0,0,0,1,0
7482378,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:07+00:00,56.10,36.0,39.0,70.54,89.0,207.53,125.70,...,2.933333,0,1.0,1,0,0,0,0,0,0
7482379,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:08+00:00,79.38,47.0,46.0,79.74,85.0,201.61,122.27,...,-6.466667,1,1.0,0,0,1,0,1,0,0
7482380,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:09+00:00,79.75,32.0,49.0,73.20,93.0,202.81,131.68,...,-0.102778,1,0.0,0,1,0,0,0,0,0


In [94]:
def calculate_hard_accel_3(x,value):
    if (x==1 and value >= 3 and value < 10):
        return 1
    else:
        return 0

In [95]:
drive_data_sort['hard_accel_3'] = np.vectorize(calculate_hard_accel_3)(drive_data_sort['accel'],drive_data_sort['calc_accel'])
drive_data_sort.head(100)

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,...,deceleration,d_lag,d_count,a_count,brake,hard_brake_10,hard_brake_3,accel,hard_accel_10,hard_accel_3
7482371,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:00+00:00,0.00,33.0,47.0,74.99,77.0,194.15,129.01,...,0,1.0,1,0,0,0,0,0,0,0
7482372,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:01+00:00,82.68,38.0,45.0,72.44,87.0,198.85,131.83,...,1,0.0,0,1,0,0,0,0,0,0
7482373,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:02+00:00,74.09,38.0,48.0,75.17,80.0,204.14,133.15,...,0,1.0,1,0,0,0,0,0,0,0
7482374,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:03+00:00,94.06,47.0,42.0,72.80,80.0,200.82,126.37,...,1,0.0,0,1,0,0,0,0,0,0
7482375,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:04+00:00,81.96,40.0,51.0,73.86,85.0,205.90,133.60,...,0,1.0,1,0,0,0,0,0,0,0
7482376,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:05+00:00,90.63,38.0,49.0,77.98,91.0,194.21,125.53,...,1,0.0,0,1,0,0,0,0,0,0
7482377,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:06+00:00,66.66,47.0,41.0,67.28,77.0,201.01,121.97,...,0,0.0,0,0,0,0,0,1,0,1
7482378,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:07+00:00,56.10,36.0,39.0,70.54,89.0,207.53,125.70,...,0,1.0,1,0,0,0,0,0,0,0
7482379,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:08+00:00,79.38,47.0,46.0,79.74,85.0,201.61,122.27,...,1,1.0,0,0,1,0,1,0,0,0
7482380,1000500,11be637c9db648899e24748c8b79da4b,2017-01-02 02:00:09+00:00,79.75,32.0,49.0,73.20,93.0,202.81,131.68,...,1,0.0,0,1,0,0,0,0,0,0


In [96]:
drive = drive_data_sort.groupby(['trip_id'])

In [97]:
f = collections.OrderedDict({'brake':['sum'],'a_count':['sum'],'d_count':['sum'],'accel':['sum'],'hard_brake_10':['sum'],'hard_brake_3':['sum'],'hard_accel_10':['sum'],'hard_accel_3':['sum'],})
drive_features = drive.agg(f)
#drive_features = drive_features.drop(columns = ['vehicle_id'],axis=1)
drive_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1708 entries, 00922df3be5a4589ab385d0c2da2dd81 to ffe2c04ae0de450cae6f41c7e2fbd86d
Data columns (total 8 columns):
(brake, sum)            1708 non-null int32
(a_count, sum)          1708 non-null int32
(d_count, sum)          1708 non-null int32
(accel, sum)            1708 non-null int32
(hard_brake_10, sum)    1708 non-null int32
(hard_brake_3, sum)     1708 non-null int32
(hard_accel_10, sum)    1708 non-null int32
(hard_accel_3, sum)     1708 non-null int32
dtypes: int32(8)
memory usage: 66.7+ KB


In [99]:
drive_features.to_csv (r'C:/Python/QuantumBlack/data/drive_features.csv',index_label=['trip_id'],header=['ft_cnt_vehicle_deaccel_val','ft_sum_time_accel_val','ft_sum_time_deaccel_val','ft_cnt_vehicle_accel_val','ft_sum_hard_brakes_10_flg_val','ft_sum_hard_brakes_3_flg_val','ft_sum_hard_accel_10_flg_val','ft_sum_hard_accel_3_flg_val'])

Working on Weather features


In [101]:
trip_data = trip_data_orig.copy()
trip_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9217528 entries, 0 to 9217527
Data columns (total 6 columns):
vehicle_id    int64
trip_id       object
datetime      datetime64[ns, UTC]
lat           float64
long          float64
velocity      float64
dtypes: datetime64[ns, UTC](1), float64(3), int64(1), object(1)
memory usage: 421.9+ MB


In [102]:
trip_data['date'] = trip_data['datetime'] - timedelta(hours=7)
trip_data['date']=np.vectorize(convert_time)(trip_data['date'])

In [126]:
def get_hour(input_time):
    date_str=str(input_time)
    return (date_str[11:13]+"Z")


In [148]:
trip_data['time'] = np.vectorize(get_hour)(trip_data['datetime'])
trip_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9217528 entries, 0 to 9217527
Data columns (total 8 columns):
vehicle_id    int64
trip_id       object
datetime      datetime64[ns, UTC]
lat           float64
long          float64
velocity      float64
date          object
time          object
dtypes: datetime64[ns, UTC](1), float64(3), int64(1), object(3)
memory usage: 562.6+ MB


In [152]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10586016 entries, 10530772 to 960615
Data columns (total 14 columns):
x                     int64
y                     int64
date                  object
time                  object
lat                   float64
lon                   float64
temperature_data      object
temperature_unit      object
precipitation_data    object
precipitation_unit    object
wind_ew_data          object
wind_ew_unit          object
wind_ns_data          object
wind_ns_unit          object
dtypes: float64(2), int64(2), object(10)
memory usage: 1.2+ GB


In [169]:
trip_weather = pd.merge(trip_data,weather_data,how='inner',left_on=['lat','long','date','time'],right_on=['lat','lon','date','time'])

In [170]:
print(trip_weather.head(50))


    vehicle_id                           trip_id                  datetime  \
0      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
1      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
2      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
3      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
4      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
5      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
6      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
7      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
8      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
9      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
10     1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
11     1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:0

In [172]:
a = trip_weather['lat'] == 32.3125
b = trip_weather['long'] == -85.062500
c = trip_weather['date'] == "2017-01-02"
d = trip_weather['time'] =='21Z'
print(trip_weather[a & b & c & d])

     vehicle_id                           trip_id                  datetime  \
0       1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
1       1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
2       1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
3       1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
4       1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
5       1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
6       1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
7       1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
8       1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
9       1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
10      1000504  192637d8a8a841dd85942a5ada12c50a 2017-01-02 21:00:00+00:00   
11      1000504  192637d8a8a841dd85942a5ada12c50a 20

In [173]:
print(trip_weather.shape)

(37819, 19)


In [174]:
trip_weather = trip_weather.sort_values(['vehicle_id','trip_id','datetime','lat','long'])

In [175]:
print(trip_weather.head(50))

       vehicle_id                           trip_id                  datetime  \
27789     1000500  228883883e1e48388427bda2b3c8315d 2017-01-02 16:00:00+00:00   
27790     1000500  228883883e1e48388427bda2b3c8315d 2017-01-02 16:00:00+00:00   
27791     1000500  228883883e1e48388427bda2b3c8315d 2017-01-02 16:00:00+00:00   
27792     1000500  228883883e1e48388427bda2b3c8315d 2017-01-02 16:00:00+00:00   
27793     1000500  228883883e1e48388427bda2b3c8315d 2017-01-02 16:00:00+00:00   
27794     1000500  228883883e1e48388427bda2b3c8315d 2017-01-02 16:00:00+00:00   
27795     1000500  228883883e1e48388427bda2b3c8315d 2017-01-02 16:00:00+00:00   
27796     1000500  228883883e1e48388427bda2b3c8315d 2017-01-02 16:00:00+00:00   
27797     1000500  228883883e1e48388427bda2b3c8315d 2017-01-02 16:00:00+00:00   
27798     1000500  228883883e1e48388427bda2b3c8315d 2017-01-02 16:00:00+00:00   
27799     1000500  228883883e1e48388427bda2b3c8315d 2017-01-02 16:00:00+00:00   
27800     1000500  228883883